In [30]:
# from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
# from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
# from langchain.prompts import example_selector
# from langchain.prompts.example_selector.base import BaseExampleSelector
# from langchain.prompts.pipeline import PipelinePromptTemplate
# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import InMemoryCache, SQLiteCache
# from langchain.llms.openai import OpenAI
# from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, load_prompt, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
            {"input": question}, 
            {"output": result.content},
        )
    print(result)
    

invoke_chain("내 이름은 정훈입니다.")

content='안녕하세요, 정훈님! 만나서 반갑습니다. 저는 여러분을 도와드리기 위해 여기 있습니다. 무엇을 도와드릴까요?'


In [31]:
invoke_chain("내 이름이 뭔가요?")

content='정훈님의 이름은 이미 말씀하셨습니다. 정훈님의 이름은 정훈입니다.'
